In [1]:
#to get the files from google drive folder
!pip install kora -q
from kora import drive
drive.link_nbs()

     |████████████████████████████████| 57 kB 1.4 MB/s 
     |████████████████████████████████| 56 kB 2.3 MB/s 
Mounted at /content/drive


In [2]:
import pandas as pd
import requests
import io
import numpy as np  
from datetime import date, timedelta
import re
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
#from github import Github
#import github
import torch
import torch.nn as nn
# Import tensor dataset & data loader
from torch.utils.data import TensorDataset, DataLoader
# Import nn.functional
import torch.nn.functional as F
import torch.optim as optim
from typing import Union, Tuple
import os
import sys
import time
from collections import OrderedDict
from sklearn.preprocessing import MinMaxScaler
from statistics import mean
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
import math
import random
import imageio
from data_preparation import create_dataset, data_preparation
from model_fit_code import fit
#from sklearn.metrics import mean_absolute_percentage_error
matplotlib.style.use('seaborn')
%matplotlib inline
#random.seed(42)
#torch.manual_seed(42)
#np.random.seed(42)

In [3]:
#generalisd implementation of GRU architecture
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim # hidden dimention
        self.n_layers = n_layers # number of layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True) # nn gru layer
        self.fc = nn.Linear(hidden_dim, output_dim) # linear layer on top of gru layer
        self.relu = nn.ReLU() # relu activation function
         
    def forward(self, x):
        weight = next(self.parameters()).data
        h = weight.new(self.n_layers, x.size(0), self.hidden_dim).zero_()
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out

In [5]:
Shortlisted_States=['Karnataka','Maharashtra','Uttar-Pradesh','Kerala','Tamil-Nadu'] #list of states 
results_gru=[]
for state in Shortlisted_States: ##state iteration
  best_models=[]
  #getting data from github
  df=pd.read_csv("https://raw.githubusercontent.com/sureshkuc/Data-Science-in-Life-Science-Project/main/Indian-States-Covid19-Datasets/"+state+".csv", parse_dates=["Date"]).drop(columns =["Unnamed: 0"])
  df = df[df["Date"] > "2020-03-10"] # selecting data from 10th March 2020 onwards
  df = df.set_index("Date")
  df = df[['Confirmed', 'Recovered', 'Deceased', 'New_Confirmerd', 'New_Deaths', 'New_Recovered']] # list of selected features
  #print(df.describe())

  time_step=[5,7,15,30] # list of time step
  Number_of_feature=[1,2,3,4,5,6] # list of feature index
  multi_feature=True
  output_dim=1
  for n_f in Number_of_feature: # feature iteration
    for t_s in time_step: # time step iteration 
      #data preprocessing
      train_loader, test_loader, scaler = data_preparation(df, scaling_range=(0,1),time_step=t_s,number_feature=n_f, response_variable_index=0,data_split_ratio=0.8, Suffle=False)
      for n_layers in range(1,3,1): #layers iteration
        for n_hidden_nodes in [1,5,8,16,32]: # hidden nodes list iteration
          
          max_epochs=25 # max epochs
          # setting seed to reproduce the results
          random.seed(42)
          torch.manual_seed(42)
          np.random.seed(42)
          #CNN model with L1 loss
          #best_model=Call_CNN_model(state,dataset=(train_loader, test_loader), lr=1e-2,criterion=nn.L1Loss(),max_epochs=max_epochs)
          GRUNet_model = GRUNet(n_f, n_hidden_nodes, output_dim, n_layers)
          #if torch.cuda.is_available():
          #stm_model = lstm_model.cuda()
          #gru_optim = optim.SGD(GRUNet_model.parameters(), lr=1e-3, momentum=0.9)
          gru_optim = optim.Adam(GRUNet_model.parameters(), lr=1e-3)
          train_losses,test_losses,best_model = fit(GRUNet_model, gru_optim,nn.L1Loss(),(train_loader, test_loader), max_epochs=max_epochs,cuda=False)
          #print(f'\nTraining took {end-start}s!')
          #plot_loss(max_epochs,train_losses,test_losses,model_name='CNN for '+state)
          GRUNet_model = GRUNet(n_f, n_hidden_nodes, output_dim, n_layers)
          GRUNet_model.load_state_dict(best_model)
          GRUNet_model.eval()
          test_x,test_y=test_loader
          predictions=GRUNet_model(test_x)
          test_y=test_y.cpu().detach().numpy()
          predictions=predictions.cpu().detach().numpy()
          mae=mean_absolute_error(test_y,predictions)
          rmse=math.sqrt(mean_squared_error(test_y,predictions))
          #mape=mean_absolute_percentage_error(test_y,predictions)
          r2s=r2_score(test_y,predictions)
          results_gru.append([state,n_f,t_s,n_layers,n_hidden_nodes,mae,rmse,r2s])
          print(state,'n_f',n_f,'t_s',t_s,'n_layers',n_layers,n_hidden_nodes,'Error',mae,rmse,r2s)

(384, 5, 1)
(384, 5, 1) (384, 1)
Epoch: 0/25  Loss: 0.267581 Test loss: 0.250573Karnataka n_f 1 t_s 5 n_layers 1 1 Error 0.5510571 0.6065956212021238 -5.60329328697985
Epoch: 0/25  Loss: 0.444396 Test loss: 1.093293Karnataka n_f 1 t_s 5 n_layers 1 5 Error 0.6523116 0.6976152927042509 -7.733614536892693
Epoch: 0/25  Loss: 0.147575 Test loss: 0.245210Karnataka n_f 1 t_s 5 n_layers 1 8 Error 0.3266558 0.3932463808481243 -1.7751822152328631
Epoch: 0/25  Loss: 0.192040 Test loss: 0.458475Karnataka n_f 1 t_s 5 n_layers 1 16 Error 0.09795287 0.11803494247955973 0.7499751115444157
Epoch: 0/25  Loss: 0.134923 Test loss: 0.417773Karnataka n_f 1 t_s 5 n_layers 1 32 Error 0.14954539 0.1721026148877546 0.46845871221696067
Epoch: 0/25  Loss: 0.628566 Test loss: 1.419832Karnataka n_f 1 t_s 5 n_layers 2 1 Error 0.67346364 0.7136362435435208 -8.139361765503137
Epoch: 0/25  Loss: 0.191334 Test loss: 0.200133Karnataka n_f 1 t_s 5 n_layers 2 5 Error 0.49242127 0.5461023867555833 -4.351926893363695
Epoch: 

KeyboardInterrupt: ignored